In [1]:
import os 
import pandas as pd
import numpy as np
import json
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "arial"
matplotlib.rcParams["pdf.fonttype"] = 42
import seaborn as sns

os.chdir("/Volumes/Kyle_T7_2/grimes_lab/analysis/"\
    "2024_01_10_multilin_grn_visualization/")

In [2]:
# Read in the connections that make up the network
path_grn_input = "/Volumes/Kyle_T7_2/grimes_lab/analysis/"\
    "2023_06_12_tea_seq_atac_processing/output/GRN/"

uconns = pd.read_feather(os.path.join(path_grn_input, 
    "all_unique_connections.fea"))

uconns = uconns.loc[~(uconns["TF_CON"] + ":" + uconns["Gene"] + ":" + \
    uconns["seqlet_idx"].astype(str)).duplicated().values]

uconns.shape

(511115, 18)

In [116]:
# Read in the cluster gex scores
path_cluster_gex_scores = "/Volumes/Kyle_T7_2/grimes_lab/analysis/"\
    "2023_06_12_tea_seq_atac_processing/output/GRN/cluster_gex_scores.csv"

gex_scores = pd.read_csv(path_cluster_gex_scores).set_index("UID")

gex_scores

,BMCP,CD127_MP,CLP1_Rrm2,eHSC,eHSC_Pcna,ERP1,ERP2,HSCP_ERP1,HSCP_HPC_Cenpf,HSCP_HPC_Hist1h2af,...,MPP5_Egr1,MPP5_Flt3,MultiLin_1,MultiLin_1_MEP,MultiLin_2_F13a1,MultiLin_2_Ms4a3,pre_MultiLin_1,pre_MultiLin_2,proNeu_1,ST_HSC
UID,,,,,,,,,,,,,,,,,,,,,
Xkr4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gm1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gm37381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rp1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Rp1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC168977.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
PISD,0.469840,0.405604,0.443472,0.498331,0.537392,0.209332,0.170386,0.392729,0.512130,0.495527,...,0.716920,0.825943,0.493832,0.601448,0.284897,0.393110,0.639836,0.594738,0.53568,0.865761
DHRSX,0.506047,0.985069,0.453410,0.498110,0.555106,0.623868,0.887142,0.419363,0.531815,0.641938,...,0.462474,0.437487,0.503296,0.604160,0.755218,0.677161,0.417228,0.392336,0.95988,0.400456


In [3]:
# Read in the cluster specific connections observed
path_grn_cluster_conns = "/Volumes/Kyle_T7_2/grimes_lab/analysis/"\
    "2023_06_12_tea_seq_atac_processing/output/GRN/cluster_grn_connections/"

cluster_files = [i for i in os.listdir(path_grn_cluster_conns) if \
    i.endswith(".fea") and not i.startswith(".")]

cluster_conns = {}
for tmp_file in cluster_files:
    tmp_conns = pd.read_feather(os.path.join(\
        path_grn_cluster_conns,
        tmp_file))
    tmp_conns = tmp_conns.loc[~(tmp_conns["TF_CON"] + ":" + \
        tmp_conns["Gene"] + ":" + tmp_conns["seqlet_idx"].astype(\
            str)).duplicated().values]
    cluster_conns[tmp_file[:-16]] = tmp_conns

In [5]:
len((cluster_conns["BMCP"]["Gene"] + ":" + cluster_conns["BMCP"]["TF_CON"]).unique())

6636

In [7]:
len((uconns["Gene"] + ":" + uconns["TF_CON"]).unique())

21961

In [144]:
# Read in the original network file (connections developed across all clusters)
with open("output/all_unique_conns_corr_0_08_re_export.cyjs", "r") as tmp_file:
    nw = json.load(tmp_file)

In [145]:
len(nw["elements"]["nodes"])

645

In [146]:
# Cytoscape helper functions
def get_node_data_df(input_network):
    return(pd.concat(\
        [pd.Series(i['data']) for i in input_network['elements']['nodes']], 
        axis=1).T.set_index('id'))

def get_node_positions_df(input_network):
    return(pd.concat([pd.Series(i['position'], name=i['data']['id']) for i in \
        input_network['elements']['nodes']], axis=1).T)

def get_edge_data_df(input_network):
    return(pd.concat(\
        [pd.Series(i['data']) for i in input_network['elements']['edges']], 
        axis=1).T.set_index('id'))

In [147]:
tot_nw_node_data = get_node_data_df(nw)
tot_nw_node_pos = get_node_positions_df(nw)
tot_nw_edge_data = get_edge_data_df(nw)
edge_conn_to_id = pd.Series(\
    tot_nw_edge_data.index.values, 
    index=(tot_nw_edge_data["source"].astype(str) + ":" + \
        tot_nw_edge_data["target"].astype(str)).values)

In [148]:
edge_conn_to_id

23241:39999    172277
23241:34833    167975
23921:41017    173208
23921:28007    161660
23921:23913    158061
                ...  
155:4253       141574
155:3665       141082
155:1769       139561
155:1553       139371
155:157        138333
Length: 19248, dtype: object

In [150]:
tmp_conns

837:20733      0.015067
837:17337      0.017399
837:27101      0.019431
837:13771      0.071709
837:1191       0.004878
                 ...   
729:39171      0.093307
729:Smim14     0.100597
729:Tap2       0.072263
729:Tbc1d9b    0.218091
729:15915      0.174200
Length: 6636, dtype: float64

In [152]:
cluster_conns["LT_HSC_Mllt3"]

array(['1043:21125', '1043:28263', '1043:34141', ..., '937:32625',
       '937:39753', '937:8989'], dtype=object)

In [159]:
path_output_files = "/Volumes/Kyle_T7_2/grimes_lab/analysis/"\
    "2024_01_10_multilin_grn_visualization/output/"\
    "cluster_specific_network_cyjs/"

for tmp_cluster in cluster_conns:
    print(f"Working on cluster {tmp_cluster}...")
    tmp_conns = cluster_conns[tmp_cluster][["Gene", "TF_CON", "con_score"]].copy()
    # Save list of TF names
    tmp_tf_list = tmp_conns["TF_CON"].unique()
    # Filter the node data to the relevant nodes
    tmp_nodes = np.unique(\
        list(tmp_conns["Gene"].values) + \
        list(tmp_conns["TF_CON"].values))
    tmp_node_data = tot_nw_node_data.loc[\
        tot_nw_node_data["name"].isin(tmp_nodes)].copy()
    map_node_name_to_id = pd.Series(\
        tmp_node_data.index.values, 
        index = tmp_node_data["name"].values)
    # Average the con_score values to get a weight for the edges
    tmp_conns["edge"] = (tmp_conns["TF_CON"] + ":" + tmp_conns["Gene"]).values
    tmp_conns = pd.pivot_table(tmp_conns, index="edge", aggfunc=np.mean)
    tmp_conns[["TF", "Gene"]] = [i.split(":") for i in tmp_conns.index.values]
    tmp_conns[["TF", "Gene"]] = tmp_conns[["TF", "Gene"]].replace(\
        map_node_name_to_id.to_dict()).values
    tmp_conns = pd.Series(\
        tmp_conns["con_score"].values, 
        index=(tmp_conns["TF"].astype(str) + ":" + \
            tmp_conns["Gene"].astype(str)).values)
    shared_conns = np.intersect1d(\
        tmp_conns.index.values, 
        edge_conn_to_id.index.values)
    tmp_conns = tmp_conns.loc[shared_conns]
    tmp_conns.index = edge_conn_to_id.loc[tmp_conns.index.values]
    # Filter the edges and attach the scores
    tmp_edges = tot_nw_edge_data.loc[tmp_conns.index.values]
    tmp_edges["edge_weight"] = 0.25 + (2*tmp_conns.values)
    # Add gex scores
    tmp_node_data["gex_score"] = gex_scores.loc[\
        tmp_node_data["name"].values, 
        tmp_cluster].values
    # Add node type
    tmp_node_data["type"] = "Gene"
    tmp_node_data.loc[tmp_node_data["name"].isin(tmp_tf_list), "type"] = "TF"


    ## Build new dictionary for json output
    # Nodes annotation
    tmp_output_nodes = []
    for i,row in tmp_node_data.reset_index().iterrows():
        tmp_output_nodes.append({\
            "data": row.to_dict(),
            "position": tot_nw_node_pos.loc[row["id"],:].to_dict(),
            "selected": False})

    # Edges annotation
    tmp_output_edges = []
    for i,row in tmp_edges.reset_index().iterrows():
        tmp_output_edges.append({\
            "data": row.to_dict(),
            "selected": False})

    # Create the network as dictionary
    tmp_out_nw = {\
        "format_version": '1.0',
        "generated_by": 'cytoscape-3.10.1',
        "target_cytoscapejs_version": '~2.1',
        "data": {\
            "shared_name": f'{tmp_cluster}__unique_connections.sif',
            "name": f'{tmp_cluster}__unique_connections.sif',
            "SUID": 128,
            "__Annotations": [],
            "selected": True},
        "elements": {\
            "nodes": tmp_output_nodes,
            "edges": tmp_output_edges}}

    # Convert and write JSON object to file
    tmp_out_file = os.path.join(\
        path_output_files,
        f"{tmp_cluster}__unique_connections_network.cyjs")
    with open(tmp_out_file, "w") as outfile: 
        json.dump(tmp_out_nw, outfile)


Working on cluster ERP2...


/var/folders/gl/6mrjx7xn0mb1m9dqk4tnl4bw0000gn/T/ipykernel_42076/3149533526.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp_conns = pd.pivot_table(tmp_conns, index="edge", aggfunc=np.mean)


Working on cluster IG2_proNeu1...


/var/folders/gl/6mrjx7xn0mb1m9dqk4tnl4bw0000gn/T/ipykernel_42076/3149533526.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp_conns = pd.pivot_table(tmp_conns, index="edge", aggfunc=np.mean)


Working on cluster HSCP_HPC_Hist1h2af...


...

/var/folders/gl/6mrjx7xn0mb1m9dqk4tnl4bw0000gn/T/ipykernel_42076/3149533526.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp_conns = pd.pivot_table(tmp_conns, index="edge", aggfunc=np.mean)


Working on cluster pre_MultiLin_2...


/var/folders/gl/6mrjx7xn0mb1m9dqk4tnl4bw0000gn/T/ipykernel_42076/3149533526.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp_conns = pd.pivot_table(tmp_conns, index="edge", aggfunc=np.mean)


Working on cluster eHSC...


/var/folders/gl/6mrjx7xn0mb1m9dqk4tnl4bw0000gn/T/ipykernel_42076/3149533526.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp_conns = pd.pivot_table(tmp_conns, index="edge", aggfunc=np.mean)


In [140]:
tmp_out_nw

{'format_version': '1.0',
 'generated_by': 'cytoscape-3.10.1',
 'target_cytoscapejs_version': '~2.1',
 'data': {'shared_name': 'BMCP__unique_connections.sif',
  'name': 'BMCP__unique_connections.sif',
  'SUID': 128,
  '__Annotations': [],
  'selected': True},
 'elements': {'nodes': [{'data': {'id': '45459',
     'shared_name': 'Ogt',
     'name': 'Ogt',
     'SUID': 45459,
     'selected': False,
     'gex_score': 0.5448156798235576,
     'type': 'Gene'},
    'position': {'x': 269.92384874006837, 'y': 160.02545244525754},
    'selected': False},
   {'data': {'id': '45321',
     'shared_name': 'F630028O10Rik',
     'name': 'F630028O10Rik',
     'SUID': 45321,
     'selected': False,
     'gex_score': 0.0988976228383672,
     'type': 'Gene'},
    'position': {'x': 72.16344224592774, 'y': -483.36490552593386},
    'selected': False},
   {'data': {'id': '45135',
     'shared_name': 'Pola1',
     'name': 'Pola1',
     'SUID': 45135,
     'selected': False,
     'gex_score': 0.37974310618262

In [138]:
tmp_edges

,source,target,shared_name,shared_interaction,name,interaction,SUID,BEND_MAP_ID,selected,edge_weight
id,,,,,,,,,,
20739,837,20733,Arnt (pd) AW112010,pd,Arnt (pd) AW112010,pd,20739,20739,False,0.280135
17343,837,17337,Arnt (pd) Abcg1,pd,Arnt (pd) Abcg1,pd,17343,17343,False,0.284799
27107,837,27101,Arnt (pd) Adgrl4,pd,Arnt (pd) Adgrl4,pd,27107,27107,False,0.288862
13777,837,13771,Arnt (pd) Angpt1,pd,Arnt (pd) Angpt1,pd,13777,13777,False,0.393418
1197,837,1191,Arnt (pd) Arl4c,pd,Arnt (pd) Arl4c,pd,1197,1197,False,0.259757
...,...,...,...,...,...,...,...,...,...,...
39193,729,39171,Zfp422 (pd) Pycard,pd,Zfp422 (pd) Pycard,pd,39193,39193,False,0.436614
30713,729,30629,Zfp422 (pd) Smim14,pd,Zfp422 (pd) Smim14,pd,30713,30713,False,0.451195
17783,729,17757,Zfp422 (pd) Tap2,pd,Zfp422 (pd) Tap2,pd,17783,17783,False,0.394526


In [72]:
tmp_nodes

array(['1700020L24Rik', '1810037I17Rik', '2610318N02Rik', '4930519L02Rik',
       '4930523C07Rik', 'AW112010', 'Abcc1', 'Abcg1', 'Abcg2', 'Abhd2',
       'Acadl', 'Adgrg3', 'Adgrl4', 'Agps', 'Alox5', 'Angpt1', 'Aprt',
       'Aqp11', 'Arap3', 'Arl4c', 'Arl6ip1', 'Arl6ip6', 'Arnt', 'Ass1',
       'Atf1', 'Atf2', 'Atf4', 'Atf6', 'Atf6b', 'Atf7', 'Atp1a3',
       'Atpif1', 'Azin1', 'B9d2', 'BC028528', 'BC035044', 'Bach1', 'Batf',
       'Bckdha', 'Bcl11a', 'Bcl2', 'Bex6', 'Bin1', 'Bola2', 'Btg2',
       'Btnl9', 'C030034L19Rik', 'C1galt1', 'Calr', 'Camk1', 'Capg',
       'Capza1', 'Carnmt1', 'Casp3', 'Cbfb', 'Ccl9', 'Ccnb1', 'Ccnb2',
       'Ccnd1', 'Cd34', 'Cd48', 'Cd52', 'Cd53', 'Cd55', 'Cd69', 'Cdc20',
       'Cdca7', 'Cdk4', 'Cebpa', 'Cebpb', 'Cebpg', 'Cenpf', 'Cenph',
       'Cep68', 'Chaf1a', 'Chdh', 'Chek1', 'Chst15', 'Ciapin1', 'Cisd1',
       'Ckap2', 'Clec12a', 'Clec4d', 'Clic1', 'Clspn', 'Cltb', 'Cnbp',
       'Cndp2', 'Coro2a', 'Cox5a', 'Cpa3', 'Creb1', 'Creb3l2', 'Crip1',
   

In [62]:
tot_nw_node_data

,shared_name,name,SUID,selected
id,,,,
45863,Prdx4,Prdx4,45863,False
45675,Gprasp2,Gprasp2,45675,False
45657,Tspan6,Tspan6,45657,False
45459,Ogt,Ogt,45459,False
45343,Dlg3,Dlg3,45343,False
...,...,...,...,...
169,Xbp1,Xbp1,169,False
165,Atf6b,Atf6b,165,False
161,Usf1,Usf1,161,False


['Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',
 'Ube2w',


In [52]:
tot_nw_edge_data

,source,target,shared_name,shared_interaction,name,interaction,SUID,BEND_MAP_ID,selected
id,,,,,,,,,
41041,23921,41017,Heyl (pd) Il27ra,pd,Heyl (pd) Il27ra,pd,41041,41041,False
28015,23921,28007,Heyl (pd) Ctnnal1,pd,Heyl (pd) Ctnnal1,pd,28015,28015,False
23923,23921,23913,Heyl (pd) Nol4l,pd,Heyl (pd) Nol4l,pd,23923,23923,False
40001,23241,39999,Tcfl5 (pd) Ckap2,pd,Tcfl5 (pd) Ckap2,pd,40001,40001,False
35383,23241,35367,Tcfl5 (pd) Ptms,pd,Tcfl5 (pd) Ptms,pd,35383,35383,False
...,...,...,...,...,...,...,...,...,...
3667,155,3665,Mlx (pd) Ndufs7,pd,Mlx (pd) Ndufs7,pd,3667,3667,False
2667,155,2665,Mlx (pd) 4930523C07Rik,pd,Mlx (pd) 4930523C07Rik,pd,2667,2667,False
1771,155,1769,Mlx (pd) Dbi,pd,Mlx (pd) Dbi,pd,1771,1771,False


In [38]:
nw['elements']['nodes']

[{'data': {'id': '45863',
   'shared_name': 'Prdx4',
   'name': 'Prdx4',
   'SUID': 45863,
   'selected': False},
  'position': {'x': -107.04763258317382, 'y': -26.405615975152614},
  'selected': False},
 {'data': {'id': '45675',
   'shared_name': 'Gprasp2',
   'name': 'Gprasp2',
   'SUID': 45675,
   'selected': False},
  'position': {'x': 121.96553002974122, 'y': 198.04223329852903},
  'selected': False},
 {'data': {'id': '45657',
   'shared_name': 'Tspan6',
   'name': 'Tspan6',
   'SUID': 45657,
   'selected': False},
  'position': {'x': 383.29559861799805, 'y': 232.64972002338254},
  'selected': False},
 {'data': {'id': '45459',
   'shared_name': 'Ogt',
   'name': 'Ogt',
   'SUID': 45459,
   'selected': False},
  'position': {'x': 269.92384874006837, 'y': 160.02545244525754},
  'selected': False},
 {'data': {'id': '45343',
   'shared_name': 'Dlg3',
   'name': 'Dlg3',
   'SUID': 45343,
   'selected': False},
  'position': {'x': 42.50948869368165, 'y': 43.893700431097386},
  'selected

,x,y
45863,-107.047633,-26.405616
45675,121.965530,198.042233
45657,383.295599,232.649720
45459,269.923849,160.025452
45343,42.509489,43.893700
...,...,...
169,-5.191935,-334.579200
165,-43.129511,-280.810828
161,9.290075,-275.452460
157,86.785287,-281.599097


In [22]:
nw['format_version']

'1.0'

In [23]:
nw['generated_by']

'cytoscape-3.10.1'

In [24]:
nw['target_cytoscapejs_version']

'~2.1'

In [32]:
nw['elements']['edges']

[{'data': {'id': '41041',
   'source': '23921',
   'target': '41017',
   'shared_name': 'Heyl (pd) Il27ra',
   'shared_interaction': 'pd',
   'name': 'Heyl (pd) Il27ra',
   'interaction': 'pd',
   'SUID': 41041,
   'BEND_MAP_ID': 41041,
   'selected': False},
  'selected': False},
 {'data': {'id': '28015',
   'source': '23921',
   'target': '28007',
   'shared_name': 'Heyl (pd) Ctnnal1',
   'shared_interaction': 'pd',
   'name': 'Heyl (pd) Ctnnal1',
   'interaction': 'pd',
   'SUID': 28015,
   'BEND_MAP_ID': 28015,
   'selected': False},
  'selected': False},
 {'data': {'id': '23923',
   'source': '23921',
   'target': '23913',
   'shared_name': 'Heyl (pd) Nol4l',
   'shared_interaction': 'pd',
   'name': 'Heyl (pd) Nol4l',
   'interaction': 'pd',
   'SUID': 23923,
   'BEND_MAP_ID': 23923,
   'selected': False},
  'selected': False},
 {'data': {'id': '40001',
   'source': '23241',
   'target': '39999',
   'shared_name': 'Tcfl5 (pd) Ckap2',
   'shared_interaction': 'pd',
   'name': 'Tcf

Examples of edges

{'data': {'id': '41041',
   'source': '23921',
   'target': '41017',
   'shared_name': 'Heyl (pd) Il27ra',
   'shared_interaction': 'pd',
   'name': 'Heyl (pd) Il27ra',
   'interaction': 'pd',
   'SUID': 41041,
   'BEND_MAP_ID': 41041,
   'selected': False},
  'selected': False},
 {'data': {'id': '28015',
   'source': '23921',
   'target': '28007',
   'shared_name': 'Heyl (pd) Ctnnal1',
   'shared_interaction': 'pd',
   'name': 'Heyl (pd) Ctnnal1',
   'interaction': 'pd',
   'SUID': 28015,
   'BEND_MAP_ID': 28015,
   'selected': False},
  'selected': False},
  ...